Churn

In [166]:
# import libraries
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import math
import matplotlib.pyplot as plt

In [167]:
# Read data
df = pd.read_json('data/mini_sparkify_event_data.json', orient='records', lines=True)
df.head(5)

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length
0,1538352117000,30,29,NextSong,Logged In,PUT,200,paid,50,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Martha Tilston,Rockpools,277.89016
1,1538352180000,9,8,NextSong,Logged In,PUT,200,free,79,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Five Iron Frenzy,Canada,236.09424
2,1538352394000,30,29,NextSong,Logged In,PUT,200,paid,51,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Adam Lambert,Time For Miracles,282.82730
3,1538352416000,9,8,NextSong,Logged In,PUT,200,free,80,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Enigma,Knocking On Forbidden Doors,262.71302
4,1538352676000,30,29,NextSong,Logged In,PUT,200,paid,52,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Daft Punk,Harder Better Faster Stronger,223.60771


In [ ]:

# drop anonymous
df = df[df['userId'] != '']

# convert userid type from string to numeric
df['userId'] = df['userId'].astype('int')

In [169]:
# custom function for lambda. convert raw numeric timestamp to datetime format.
def convert_ts(x):
    dt = datetime.utcfromtimestamp(int(x) / 1000)
    return dt

In [171]:
# Add event log datetime column. (human readable format) based on ts column
df['datetime'] = df.ts.apply(convert_ts)

# Add user registration datetime column. For later tenure calculation.
df['reg'] = df['registration'].apply(convert_ts)

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length,datetime
0,1538352117000,30,29,NextSong,Logged In,PUT,200,paid,50,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Martha Tilston,Rockpools,277.89016,2018-10-01 00:01:57
1,1538352180000,9,8,NextSong,Logged In,PUT,200,free,79,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Five Iron Frenzy,Canada,236.09424,2018-10-01 00:03:00
2,1538352394000,30,29,NextSong,Logged In,PUT,200,paid,51,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Adam Lambert,Time For Miracles,282.82730,2018-10-01 00:06:34
3,1538352416000,9,8,NextSong,Logged In,PUT,200,free,80,"Boston-Cambridge-Newton, MA-NH","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Long,Micah,1.538332e+12,M,Enigma,Knocking On Forbidden Doors,262.71302,2018-10-01 00:06:56
4,1538352676000,30,29,NextSong,Logged In,PUT,200,paid,52,"Bakersfield, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Freeman,Colin,1.538173e+12,M,Daft Punk,Harder Better Faster Stronger,223.60771,2018-10-01 00:11:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286495,1543622240000,,500,Home,Logged Out,GET,200,paid,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-30 23:57:20
286496,1543622241000,,500,Login,Logged Out,PUT,307,paid,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-30 23:57:21
286497,1543622248000,300011,500,Home,Logged In,GET,200,paid,43,"New York-Newark-Jersey City, NY-NJ-PA",Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,House,Emilia,1.538337e+12,F,NaN,NaN,NaN,2018-11-30 23:57:28
286498,1543622398000,300011,500,About,Logged In,GET,200,paid,44,"New York-Newark-Jersey City, NY-NJ-PA",Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,House,Emilia,1.538337e+12,F,NaN,NaN,NaN,2018-11-30 23:59:58


In [175]:
df.dtypes

ts                        int64
userId                    int64
sessionId                 int64
page                     object
auth                     object
method                   object
status                    int64
level                    object
itemInSession             int64
location                 object
userAgent                object
lastName                 object
firstName                object
registration            float64
gender                   object
artist                   object
song                     object
length                  float64
datetime         datetime64[ns]
reg              datetime64[ns]
dtype: object

In [176]:
# Check application devices used

df['userAgent'].value_counts()

"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"                                22751
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"                        19611
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"                     18448
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"                     17348
Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0                                                                       16700
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"                                15395
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/53

In [177]:
# Check gender percentage

df['gender'].value_counts() / df['gender'].value_counts().sum()

F    0.555728
M    0.444272
Name: gender, dtype: float64

In [178]:
# Check http status code percentage

df['status'].value_counts() / df['status'].value_counts().sum()

200    0.915745
307    0.083350
404    0.000906
Name: status, dtype: float64

In [179]:
# Number of all distinct users

n_users = df['userId'].nunique()
n_users

225

In [195]:
# Number of 'paid' level users

n_paid_users = df[df['level'] == 'paid']['userId'].nunique()

165

In [199]:
# List of 'paid' users IDs

paid_user_ids = df[df['level'] == 'paid']['userId'].value_counts().index.to_list()

print(n_paid_users)
paid_user_ids

165


[39,
 92,
 140,
 300011,
 124,
 300021,
 300017,
 42,
 85,
 6,
 29,
 300015,
 9,
 100,
 91,
 54,
 126,
 118,
 74,
 51,
 98,
 136,
 65,
 4,
 81,
 15,
 132,
 142,
 41,
 138,
 88,
 300023,
 200004,
 127,
 101,
 300001,
 300018,
 128,
 200008,
 115,
 96,
 97,
 60,
 58,
 300002,
 62,
 20,
 25,
 120,
 113,
 104,
 131,
 200023,
 300009,
 300008,
 70,
 53,
 37,
 45,
 33,
 300025,
 82,
 38,
 14,
 152,
 114,
 145,
 69,
 95,
 61,
 200021,
 28,
 83,
 77,
 153,
 67,
 100013,
 103,
 17,
 52,
 35,
 46,
 300019,
 75,
 40,
 141,
 49,
 2,
 105,
 106,
 100008,
 36,
 155,
 66,
 10,
 109,
 300012,
 300016,
 111,
 100018,
 89,
 59,
 30,
 16,
 12,
 86,
 23,
 21,
 121,
 108,
 100004,
 200009,
 100016,
 56,
 100007,
 18,
 44,
 99,
 147,
 100023,
 13,
 300022,
 73,
 200025,
 200019,
 139,
 100025,
 300013,
 200017,
 100015,
 129,
 200011,
 200002,
 300014,
 200020,
 300005,
 100009,
 100014,
 300020,
 19,
 100012,
 3,
 93,
 71,
 11,
 100002,
 200024,
 200014,
 151,
 50,
 200003,
 300006,
 87,
 300004,
 300007,


In [198]:
# List of 'free' users IDs

free_user_ids = df[~df['userId'].isin(paid_user_ids)]['userId'].value_counts().index.to_list()

n_free_users = len(free_user_ids)
print(n_free_users)

free_user_ids

60


[100022,
 146,
 200006,
 148,
 200018,
 80,
 200022,
 117,
 100010,
 300010,
 200015,
 8,
 78,
 100021,
 200010,
 107,
 112,
 27,
 76,
 200016,
 47,
 43,
 149,
 110,
 150,
 119,
 5,
 100005,
 102,
 7,
 123,
 94,
 100001,
 200001,
 143,
 144,
 154,
 72,
 57,
 63,
 84,
 100019,
 200012,
 116,
 100003,
 100017,
 34,
 64,
 134,
 68,
 90,
 100006,
 133,
 22,
 100024,
 300003,
 100011,
 125,
 156,
 135]

In [186]:
# Spot Check: Number of 'authentication' values check

df['auth'].value_counts()

Logged In    278102
Cancelled        52
Name: auth, dtype: int64

In [201]:
# List of churn users

churn_user_ids = df[df['auth'] == 'Cancelled']['userId'].value_counts().index.to_list()

n_total_churn = len(churn_user_ids)

print(n_total_churn)

52


In [188]:
# Number of churn users in different subscription levels

df[df['auth'] == 'Cancelled'].groupby('level').size()

level
free    21
paid    31
dtype: int64

In [202]:
# Overall Total Churn Rate.
# Total number of churn users / total number of users

n_total_churn / n_users

0.2311111111111111

In [203]:
# Free users churn
# total number of churn users who are 'free-tier' / total number of free users

df[df['auth'] == 'Cancelled'].groupby('level').size()['free'] / n_free_users

0.35

In [204]:
# paid users churn
# total number of churn users who are  'paid-tier' / total number of paid users

df[df['auth'] == 'Cancelled'].groupby('level').size()['paid'] / n_paid_users

0.18787878787878787

In [207]:
# Spot Check: earliest date and latest event date in the dataset
print(np.min(df['datetime']))
print(np.max(df['datetime']))

# Spot Check: Date range in the dataset
print(np.max(df['datetime']) - np.min(df['datetime']))

# Spot Check: earliest date and latest user registration date in the dataset
print(np.min(df['reg']))
print(np.max(df['reg']))

2018-10-01 00:01:57
2018-12-03 01:11:16
63 days 01:09:19
2018-03-18 13:44:35
2018-11-26 15:49:14


## Features Engineering (Metrics Design)

In [230]:
# Custom lambda function. Convert useragent to device


def convert_ua(x):
    user_os = x.lower().split('/')[1]

    if 'windows' in user_os:
        x = 'PC'
    elif 'macintosh' in user_os:
        x = 'Mac'
    elif 'iphone' in user_os:
        x = 'iPhone'
    elif 'ipad' in user_os:
        x = 'iPad'
    elif 'linux' in user_os:
        x = 'Linux'
    else:
        x = 'Others'

    return x


df['device'] = df['userAgent'].apply(convert_ua)

/var/folders/z2/4x1xtsmd40sfmyp2ftrfbkfh0000gn/T/ipykernel_63451/314438376.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['device'] = df['userAgent'].apply(convert_ua)


In [233]:
df['location'].value_counts()

Los Angeles-Long Beach-Anaheim, CA        30131
New York-Newark-Jersey City, NY-NJ-PA     23684
Boston-Cambridge-Newton, MA-NH            13873
Houston-The Woodlands-Sugar Land, TX       9499
Charlotte-Concord-Gastonia, NC-SC          7780
                                          ...  
Deltona-Daytona Beach-Ormond Beach, FL       73
Memphis, TN-MS-AR                            62
Albany, OR                                   23
Corpus Christi, TX                           11
Scranton--Wilkes-Barre--Hazleton, PA          6
Name: location, Length: 114, dtype: int64

In [261]:
# Custom lambda function. Convert location to state

# https://en.wikipedia.org/wiki/List_of_regions_of_the_United_States#/media/File:Census_Regions_and_Division_of_the_United_States.svg


def convert_location(x):
    west = ['wa', 'or', 'ca', 'az', 'nm', 'co', 'wy', 'mt', 'id', 'nv', 'ut', 'ak']
    midwest = ['nd', 'sd', 'ne', 'ks', 'mn', 'ia', 'mo', 'wi', 'il', 'in', 'mi', 'oh']
    northeast = ['pa', 'ny', 'nj', 'vt', 'ma', 'ct', 'nh', 'ri', 'me']
    south = ['tx', 'ok', 'ar', 'la', 'ms', 'ky', 'tn', 'al', 'ga', 'fl', 'wv', 'va', 'nc', 'sc', 'dc', 'md', 'de']

    location = x.lower().split(',')[1][1:3]

    if location in west:
        region = 'West'
    elif location in midwest:
        region = 'Midwest'
    elif location in northeast:
        region = 'Northeast'
    elif location in south:
        region = 'South'
    else:
        # print(location)
        region = 'Others'

    return region


df['region'] = df['location'].apply(convert_location)

/var/folders/z2/4x1xtsmd40sfmyp2ftrfbkfh0000gn/T/ipykernel_63451/637794245.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['region'] = df['location'].apply(convert_location)


In [270]:
df['region'].value_counts() / np.sum(df['region'].value_counts())

South        0.368605
West         0.260410
Northeast    0.255732
Midwest      0.115253
Name: region, dtype: float64

In [275]:
# Custom lambda function: compute tenure

def compute_tenure(x):
    return x

df['reg'][:5]

0   2018-09-28 22:22:42
1   2018-09-30 18:20:30
2   2018-09-28 22:22:42
3   2018-09-30 18:20:30
4   2018-09-28 22:22:42
Name: reg, dtype: datetime64[ns]